In [1]:
import math
import pylab
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torch.utils import data
import argparse
import torch
import torch.utils.data as data_utils
import torch.optim as optim
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F

seed = 2021
torch.manual_seed(seed)
torch.cuda.manual_seed(0)
np.random.seed(seed)

torch.backends.cudnn.deterministic = True

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)



class Dataset(data.Dataset):
    def __init__(self, X1, Y1):
        self.X1 = X1
        self.Y1 = Y1

    def __len__(self):
        return len(self.X1)

    def __getitem__(self, index):
        x = self.X1[index]
        y1 = self.Y1[index]
        return x, y1

In [2]:
from collections import OrderedDict
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)
    if type(m) == nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.L = 128
        self.D = 64
        self.K = 1
        self.feature_size = 24
        self.shared_layer_size = 128

        self.fc = nn.Sequential(
            nn.Conv1d(1, 128, kernel_size=(2,1),padding=1,stride=1),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Conv1d(128, 32, kernel_size=(2,1),padding=1,stride=1),
            nn.ReLU()
          )
        self.fc2 = nn.Sequential(
            nn.Linear(18816, 512),
            nn.BatchNorm1d(512,affine=False),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )


    def forward(self, x):
        
        Y_prob=self.fc(x)
        Y_prob=self.fc2(Y_prob.reshape(Y_prob.size(0),-1))
        
        return Y_prob

In [3]:
from torch.optim.lr_scheduler import _LRScheduler
class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]

In [4]:
def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler

In [5]:
xTrain=np.load("../extracted_features/0/GRP-3/input_features_train_grp3.npy")
yTrain=np.load("../extracted_features/0/GRP-3/labels_train_grp3.npy")

xTest=np.load("../extracted_features/0/GRP-3/input_features_test_grp3.npy")
yTest=np.load("../extracted_features/0/GRP-3/labels_test_grp3.npy")


xVal=np.load("../extracted_features/0/GRP-3/input_features_val_grp3.npy")
yVal=np.load("../extracted_features/0/GRP-3/labels_val_grp3.npy")

train_features=np.load("../extracted_features/bl/0/GRP-3/input_features_train_grp3.npy")
labels_train=np.load("../extracted_features/bl/0/GRP-3/labels_train_grp3.npy")

test_features=np.load("../extracted_features/bl/0/GRP-3/input_features_test_grp3.npy")
labels_test=np.load("../extracted_features/bl/0/GRP-3/labels_test_grp3.npy")

val_features=np.load("../extracted_features/bl/0/GRP-3/input_features_val_grp3.npy")
labels_val=np.load("../extracted_features/bl/0/GRP-3/labels_val_grp3.npy")


In [6]:
import os

def readFeatures(path):
    features=np.array([])
    start=0
    for file in os.listdir(path):
        d = os.path.join(path, file)
        datafile=d+"/features.npy"
        try:
            temp=np.load(datafile)
#             print(datafile)
#             print(temp.shape)
            if(start==0):
                features=temp
                if(d[-10]=='S'):
                    label=np.ones(len(temp))
                else:
                    label=np.zeros(len(temp))
                start=1
            else:
                features=np.vstack((features,temp))
                if(d[-10]=='S'):
                    label=np.append(label,np.ones(len(temp)))
                else:
                    label=np.append(label,np.zeros(len(temp)))
        except IOError:
            print('file not in Scripted')

            
    return features,label

def load_data(path,mode):
    if mode == "test":
        xTest,yTest=readFeatures(path+"/"+mode)
        xTest = xTest.reshape(len(xTest),1,19,24)
        return Dataset(xTest,yTest)
        
    elif mode == "train":
        xTrain,yTrain=readFeatures(path+"/"+mode)
        xTrain = xTrain.reshape(len(xTrain),1,19,24)
        return Dataset(xTrain,yTrain)
        
    elif mode == "val":
        xVal,yVal=readFeatures(path+"/"+mode)
        xVal = xVal.reshape(len(xVal),1,19,24)
        return Dataset(xVal,yVal)
    else:
        print("Mode not defined")
        return

In [7]:
print(labels_train.shape,labels_train)

(497,) [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1

In [8]:
print(xTrain.shape,yTrain.shape)
print(xTest.shape,yTest.shape)
print(xVal.shape,yVal.shape)

(512, 19, 24) (512,)
(122, 19, 24) (122,)
(118, 19, 24) (118,)


In [9]:
print(train_features.shape,labels_train.shape)
print(test_features.shape,labels_test.shape)
print(val_features.shape,labels_val.shape)

(497, 19, 24) (497,)
(119, 19, 24) (119,)
(110, 19, 24) (110,)


In [10]:
train_features=train_features.reshape(len(train_features),1,19,24)
test_features=test_features.reshape(len(test_features),1,19,24)
val_features=val_features.reshape(len(val_features),1,19,24)

xTrain = xTrain.reshape(len(xTrain),1,19,24)
xTest = xTest.reshape(len(xTest),1,19,24)
xVal = xVal.reshape(len(xVal),1,19,24)

In [11]:
# xTrain=np.append(train_features,xTrain,axis=0)
# xTest=np.append(test_features,xTest,axis=0)
# xVal=np.append(val_features,xVal,axis=0)
# yTrain=np.append(labels_train,yTrain,axis=0)
# yTest=np.append(labels_test,yTest,axis=0)
# yVal=np.append(labels_val,yVal,axis=0)

In [12]:
batch_size=20

train_data = load_data("../Participant_wise",'train')
test_data=load_data("../Participant_wise",'test')
val_data=load_data("../Participant_wise",'val')

# train_data = Dataset(train_features, labels_train)
# test_data=Dataset(test_features,labels_test)
# val_data=Dataset(val_features,labels_val)

train_data_loader = DataLoader(train_data,worker_init_fn=seed_worker, shuffle=True, batch_size=batch_size)
val_data_loader=DataLoader(val_data,worker_init_fn=seed_worker,shuffle=True,batch_size=batch_size)
test_data_loader=DataLoader(test_data,worker_init_fn=seed_worker,shuffle=True,batch_size=batch_size)

In [13]:
for batch in val_data_loader:
    print(batch[1].shape)
    break
print(len(val_data_loader))

torch.Size([20])
6


In [14]:
feature_size = 24
shared_layer_size = 512
LR = 0.0001
epoch = 75
model=Classifier()
model.cuda()
iterations_per_epoch = len(train_data_loader)
model.apply(init_weights)
loss_func = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
sched = CyclicLR(optimizer, cosine(t_max=iterations_per_epoch * 2, eta_min=LR/100))

In [15]:
# from torchsummary import summary
# summary(model.cuda(),(1,19,24))
# torch.save(model, "modelarchitect_final.pt")

In [16]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
best_acc1 = 0
modelname=[]
truth=[]
preds=[]
best_model = "./saved_models/best_model_cnn_raw"
for it in range(epoch+1):
    total=len(train_data_loader)*batch_size
    train_loss = 0.
    model.train()
    for minibatch in train_data_loader:
        X, Y1  = minibatch
        X=X.cuda()
        Y1=Y1.cuda()
        output = model(X.float())
        output=output.squeeze(1)
        loss = loss_func(output, Y1.float())
        Y_hat1 = torch.ge(output, 0.5).float()
        train_loss += loss.item()
        truth.extend(Y1.tolist())
        preds.extend(Y_hat1.tolist())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        sched.step()
    trainacc1=f1_score(truth,preds)
    train_loss /= total
    print("EPOCH ",it)
    print('Train : Loss: {:.4f}, Train acc1 : {:.4f}'.format(train_loss,trainacc1))
    
    count=0
    val_loss= 0.
    truth=[]
    preds=[]
    total=len(val_data_loader)*batch_size
    model.eval()    
    for minibatch in val_data_loader:
        X_valid, Y1_valid  = minibatch
        X_valid=X_valid.cuda()
        Y1_valid=Y1_valid.cuda()
        output_val = model(X_valid.float())
        output_val=output_val.squeeze(1)
        loss = loss_func(output_val, Y1_valid.float())
        Y_hat1_val = torch.ge(output_val, 0.5).float()
        val_loss += loss.item()
        truth.extend(Y1_valid.tolist())
        preds.extend(Y_hat1_val.tolist())
    valacc1=f1_score(truth,preds)
    val_loss /= total
    print('Val : Loss: {:.4f}, Val acc1 : {:.4f}'.format(val_loss,valacc1))
    if valacc1 >= best_acc1:
        best_acc1 = valacc1
        best_state = model.state_dict()
        torch.save(best_state, best_model+'_epoch_'+str(epoch)+".pth")
        modelname.append(best_model+'_epoch_'+str(epoch)+".pth")
        print('Best validation accuracy1 ', best_acc1)
        
    count=0
    test_loss= 0.
    truth=[]
    preds=[]
    total=len(test_data_loader)*batch_size    
    for minibatch in test_data_loader:
        X_valid, Y1_valid  = minibatch
        X_valid=X_valid.cuda()
        Y1_valid=Y1_valid.cuda()
        output_val = model(X_valid.float())
        output_val=output_val.squeeze(1)
        loss = loss_func(output_val, Y1_valid.float())
        Y_hat1_val = torch.ge(output_val, 0.5).float()
        test_loss += loss.item()
        truth.extend(Y1_valid.tolist())
        preds.extend(Y_hat1_val.tolist())
    valacc1=f1_score(truth,preds)
    test_loss /= total
    print('test : Loss: {:.4f}, Val acc1 : {:.4f}'.format(val_loss,valacc1))

EPOCH  0
Train : Loss: 0.0367, Train acc1 : 0.5868
Val : Loss: 0.0342, Val acc1 : 0.5865
Best validation accuracy1  0.5864661654135338
test : Loss: 0.0342, Val acc1 : 0.6573
EPOCH  1
Train : Loss: 0.0315, Train acc1 : 0.5889
Val : Loss: 0.0355, Val acc1 : 0.3562
test : Loss: 0.0355, Val acc1 : 0.3056
EPOCH  2
Train : Loss: 0.0307, Train acc1 : 0.5497
Val : Loss: 0.0359, Val acc1 : 0.4583
test : Loss: 0.0359, Val acc1 : 0.6095
EPOCH  3
Train : Loss: 0.0288, Train acc1 : 0.6331
Val : Loss: 0.0360, Val acc1 : 0.4416
test : Loss: 0.0360, Val acc1 : 0.1481
EPOCH  4
Train : Loss: 0.0295, Train acc1 : 0.5178
Val : Loss: 0.0364, Val acc1 : 0.4330
test : Loss: 0.0364, Val acc1 : 0.4731
EPOCH  5
Train : Loss: 0.0276, Train acc1 : 0.6352
Val : Loss: 0.0371, Val acc1 : 0.4898
test : Loss: 0.0371, Val acc1 : 0.2947
EPOCH  6
Train : Loss: 0.0266, Train acc1 : 0.5904
Val : Loss: 0.0393, Val acc1 : 0.4752
test : Loss: 0.0393, Val acc1 : 0.4242
EPOCH  7
Train : Loss: 0.0252, Train acc1 : 0.6413
Val : L

EPOCH  65
Train : Loss: 0.0057, Train acc1 : 0.8836
Val : Loss: 0.0820, Val acc1 : 0.4854
test : Loss: 0.0820, Val acc1 : 0.4486
EPOCH  66
Train : Loss: 0.0047, Train acc1 : 0.8767
Val : Loss: 0.0758, Val acc1 : 0.5000
test : Loss: 0.0758, Val acc1 : 0.5812
EPOCH  67
Train : Loss: 0.0041, Train acc1 : 0.9106
Val : Loss: 0.0704, Val acc1 : 0.5098
test : Loss: 0.0704, Val acc1 : 0.4630
EPOCH  68
Train : Loss: 0.0046, Train acc1 : 0.8889
Val : Loss: 0.0688, Val acc1 : 0.5294
test : Loss: 0.0688, Val acc1 : 0.4200
EPOCH  69
Train : Loss: 0.0045, Train acc1 : 0.8765
Val : Loss: 0.0627, Val acc1 : 0.5200
test : Loss: 0.0627, Val acc1 : 0.4400
EPOCH  70
Train : Loss: 0.0049, Train acc1 : 0.8759
Val : Loss: 0.0721, Val acc1 : 0.5094
test : Loss: 0.0721, Val acc1 : 0.5042
EPOCH  71
Train : Loss: 0.0045, Train acc1 : 0.8769
Val : Loss: 0.0757, Val acc1 : 0.5333
test : Loss: 0.0757, Val acc1 : 0.5321
EPOCH  72
Train : Loss: 0.0038, Train acc1 : 0.9026
Val : Loss: 0.0690, Val acc1 : 0.5000
test : 

In [17]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

modeltest=Classifier()
best_state=torch.load(modelname[-1])
modeltest.load_state_dict(best_state)
modeltest.cuda()
modeltest.eval()
truth=[]
preds=[]
for minibatch in test_data_loader:
            X_test, Y1_test  = minibatch
            X_test=X_test.cuda()
            Y1_test=Y1_test.cuda()
            output_test = modeltest(X_test.float())
            output_test=output_test.squeeze(1)
            prediction = torch.ge(output_test, 0.5).float()
            truth.extend(Y1_test.tolist())
            preds.extend(prediction.tolist())
acc=accuracy_score(truth,preds)
# print(truth,preds)
tn, fp, fn, tp = confusion_matrix(truth, preds).ravel()
f1score=f1_score(truth, preds)
precision=precision_score(truth, preds)
recall=recall_score(truth,preds)
roc=roc_auc_score(truth,preds)
specificity=tn/(tn+fp)

print('{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(acc,f1score,precision,recall,roc,specificity))


0.58 0.66 0.55 0.81 0.59 0.37


In [18]:
testdataset = load_data("../Participant_wise_win-1",'test')

test_data_loader = DataLoader(testdataset,worker_init_fn=seed_worker,shuffle=True,batch_size=batch_size)

modeltest=Classifier()
best_state=torch.load(modelname[-1])
modeltest.load_state_dict(best_state)
modeltest.cuda()
modeltest.eval()
truth=[]
preds=[]
for minibatch in test_data_loader:
            X_test, Y1_test  = minibatch
            X_test=X_test.cuda()
            Y1_test=Y1_test.cuda()
            output_test = modeltest(X_test.float())
            output_test=output_test.squeeze(1)
            prediction = torch.ge(output_test, 0.5).float()
            truth.extend(Y1_test.tolist())
            preds.extend(prediction.tolist())
acc=accuracy_score(truth,preds)
# print(truth,preds)
tn, fp, fn, tp = confusion_matrix(truth, preds).ravel()
f1score=f1_score(truth, preds)
precision=precision_score(truth, preds)
recall=recall_score(truth,preds)
roc=roc_auc_score(truth,preds)
specificity=tn/(tn+fp)

print('{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(acc,f1score,precision,recall,roc,specificity))


0.56 0.62 0.54 0.72 0.56 0.40


In [19]:
testdataset = load_data("../Participant_wise_win-2",'test')

test_data_loader = DataLoader(testdataset,worker_init_fn=seed_worker,shuffle=True,batch_size=batch_size)

modeltest=Classifier()
best_state=torch.load(modelname[-1])
modeltest.load_state_dict(best_state)
modeltest.cuda()
modeltest.eval()
truth=[]
preds=[]
for minibatch in test_data_loader:
            X_test, Y1_test  = minibatch
            X_test=X_test.cuda()
            Y1_test=Y1_test.cuda()
            output_test = modeltest(X_test.float())
            output_test=output_test.squeeze(1)
            prediction = torch.ge(output_test, 0.5).float()
            truth.extend(Y1_test.tolist())
            preds.extend(prediction.tolist())
acc=accuracy_score(truth,preds)
# print(truth,preds)
tn, fp, fn, tp = confusion_matrix(truth, preds).ravel()
f1score=f1_score(truth, preds)
precision=precision_score(truth, preds)
recall=recall_score(truth,preds)
roc=roc_auc_score(truth,preds)
specificity=tn/(tn+fp)

print('{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(acc,f1score,precision,recall,roc,specificity))


0.61 0.69 0.57 0.90 0.61 0.33
